In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# خوارزمية التحسين التقريبي الكمومي

*تقدير الاستخدام: 22 دقيقة على معالج Heron r3 (ملاحظة: هذا تقدير فحسب. قد يختلف وقت التشغيل الفعلي.)*
## الخلفية النظرية
يوضح هذا البرنامج التعليمي كيفية تنفيذ **خوارزمية التحسين التقريبي الكمومي (QAOA)** – وهي طريقة تكرارية هجينة (كمومية-كلاسيكية) – في سياق أنماط Qiskit. ستحل أولاً مسألة **القطع الأقصى** (أو **Max-Cut**) لرسم بياني صغير، ثم ستتعلم كيفية تنفيذه على نطاق الاستخدام العملي. ينبغي أن تُنفَّذ جميع عمليات تشغيل الأجهزة في هذا البرنامج التعليمي ضمن الحد الزمني المتاح للخطة المفتوحة المتاحة مجاناً.

مسألة القطع الأقصى هي مسألة تحسين يصعب حلها (وبصورة أدق، هي مسألة *NP-hard*) ولها تطبيقات عديدة في التجميع وعلم الشبكات والفيزياء الإحصائية. يأخذ هذا البرنامج التعليمي رسماً بيانياً من عقد مرتبطة بحواف، ويهدف إلى تقسيم العقد إلى مجموعتين بحيث يكون عدد الحواف التي يقطعها هذا التقسيم أقصى ما يمكن.

![توضيح مسألة القطع الأقصى](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## المتطلبات
قبل البدء في هذا البرنامج التعليمي، تأكد من تثبيت ما يلي:
- Qiskit SDK الإصدار v1.0 أو أحدث، مع دعم [التصور المرئي](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime الإصدار v0.22 أو أحدث (`pip install qiskit-ibm-runtime`)

بالإضافة إلى ذلك، ستحتاج إلى صلاحية الوصول إلى نسخة على [IBM Quantum Platform](/guides/cloud-setup). لاحظ أن هذا البرنامج التعليمي لا يمكن تنفيذه على الخطة المفتوحة، إذ يشغّل أحمال عمل باستخدام [الجلسات](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session)، المتاحة فقط ضمن الخطة المميزة.
## الإعداد

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## الجزء الأول: QAOA على نطاق صغير
يستخدم الجزء الأول من هذا البرنامج التعليمي مسألة قطع أقصى صغيرة النطاق لتوضيح خطوات حل مسألة تحسين باستخدام حاسوب كمومي.

لإعطاء بعض السياق قبل تحويل هذه المسألة إلى خوارزمية كمومية، يمكن فهم كيفية تحوّل مسألة القطع الأقصى إلى مسألة تحسين تركيبي كلاسيكية بالنظر أولاً في تقليل دالة $f(x)$

$$
\min_{x\in {0, 1}^n}f(x),
$$

حيث المدخل $x$ هو متجه تتوافق مكوناته مع كل عقدة في الرسم البياني. بعد ذلك، يُقيَّد كل مكون من هذه المكونات ليكون إما $0$ أو $1$ (يمثلان الإدراج أو عدم الإدراج في القطع). يستخدم هذا المثال الصغير رسماً بيانياً بـ $n=5$ عقد.

يمكن كتابة دالة لزوج من العقد $i,j$ تشير إلى ما إذا كان الحافة المقابلة $(i,j)$ موجودة في القطع. على سبيل المثال، الدالة $x_i + x_j - 2 x_i x_j$ تساوي 1 فقط إذا كانت إحدى القيمتين $x_i$ أو $x_j$ تساوي 1 (مما يعني أن الحافة موجودة في القطع)، وتساوي صفراً في الحالات الأخرى. يمكن صياغة مسألة تعظيم الحواف في القطع على النحو التالي

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

والتي يمكن إعادة كتابتها على شكل تقليل كما يلي

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

الحد الأدنى لـ $f(x)$ في هذه الحالة يتحقق حين يكون عدد الحواف التي يقطعها القطع أكبر ما يمكن. كما ترى، لا يوجد حتى الآن أي ارتباط بالحوسبة الكمومية. تحتاج إلى إعادة صياغة هذه المسألة بشكل يستطيع الحاسوب الكمومي فهمه.
ابدأ بتهيئة مسألتك بإنشاء رسم بياني بـ $n=5$ عقد.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![ناتج خلية الكود السابقة](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### الخطوة 1: تحويل المدخلات الكلاسيكية إلى مسألة كمومية
الخطوة الأولى في النمط هي تحويل المسألة الكلاسيكية (الرسم البياني) إلى **دوائر** و**عوامل** كمومية. لتحقيق ذلك، هناك ثلاث خطوات رئيسية:

1. استخدام سلسلة من إعادات الصياغة الرياضية لتمثيل المسألة باستخدام تدوين مسائل التحسين الثنائي التربيعي غير المقيّد (QUBO).
2. إعادة كتابة مسألة التحسين على شكل هاميلتوني تتوافق حالته الأساسية مع الحل الذي يقلل دالة التكلفة.
3. إنشاء دائرة كمومية لتهيئة الحالة الأساسية لهذا الهاميلتوني عبر عملية مشابهة للتلدين الكمومي.

**ملاحظة:** في منهجية QAOA، الهدف النهائي هو الحصول على عامل (**هاميلتوني**) يمثل **دالة التكلفة** لخوارزميتنا الهجينة، فضلاً عن دائرة ذات معاملات (**Ansatz**) تمثل الحالات الكمومية التي تُعدّ حلولاً مرشحة للمسألة. يمكن أخذ عينات من هذه الحالات المرشحة ثم تقييمها باستخدام دالة التكلفة.

#### الرسم البياني &rarr; مسألة التحسين
الخطوة الأولى في التحويل هي تغيير التدوين؛ يعبّر ما يلي عن المسألة بتدوين QUBO:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

حيث $Q$ هي مصفوفة $n\times n$ من الأعداد الحقيقية، و$n$ يتوافق مع عدد العقد في رسمك البياني، و$x$ هو متجه المتغيرات الثنائية المُعرَّف أعلاه، و$x^T$ يشير إلى منقول المتجه $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### مسألة التحسين &rarr; الهاميلتوني
يمكنك بعد ذلك إعادة صياغة مسألة QUBO على شكل **هاميلتوني** (أي مصفوفة تمثل طاقة نظام ما):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**خطوات إعادة الصياغة من مسألة QAOA إلى الهاميلتوني**
</summary>

لتوضيح كيفية إعادة كتابة مسألة QAOA بهذه الطريقة، استبدل أولاً المتغيرات الثنائية $x_i$ بمجموعة جديدة من المتغيرات $z_i\in{-1, 1}$ عبر

$$
x_i = \frac{1-z_i}{2}.
$$

يمكن هنا ملاحظة أنه إذا كانت $x_i$ تساوي $0$، فإن $z_i$ يجب أن تساوي $1$. حين يُعوَّض $z_i$ عن $x_i$ في مسألة التحسين ($x^TQx$)، يمكن الحصول على صياغة مكافئة.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

الآن إذا عرّفنا $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$، وأسقطنا عامل الضرب المسبق والحد الثابت $n^2$، نصل إلى الصياغتين المتكافئتين لمسألة التحسين ذاتها.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

هنا، $b$ يعتمد على $Q$. لاحظ أننا للحصول على $z^TQz + b^Tz$ أسقطنا عامل 1/4 والإزاحة الثابتة $n^2$ اللتين لا تؤثران على عملية التحسين.

الآن، للحصول على صياغة كمومية للمسألة، نرفع متغيرات $z_i$ إلى مصفوفة Pauli $Z$، مثل مصفوفة $2\times 2$ من الشكل

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

حين تُعوَّض هذه المصفوفات في مسألة التحسين أعلاه، نحصل على الهاميلتوني التالي

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*تذكّر أيضاً أن مصفوفات $Z$ مُضمَّنة في فضاء العمليات الحسابية للحاسوب الكمومي، أي فضاء هيلبرت بحجم $2^n\times 2^n$. لذلك، ينبغي فهم المصطلحات مثل $Z_iZ_j$ على أنها حاصل الضرب التنسوري $Z_i\otimes Z_j$ المُضمَّن في فضاء هيلبرت $2^n\times 2^n$. فمثلاً، في مسألة بخمسة متغيرات قرار، يُفهم الحد $Z_1Z_3$ على أنه $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ حيث $I$ هي مصفوفة الهوية $2\times 2$.*
</details>

يُسمى هذا الهاميلتوني **هاميلتوني دالة التكلفة**. ويتميز بأن حالته الأساسية تتوافق مع الحل الذي **يقلل دالة التكلفة $f(x)$**.
وبالتالي، لحل مسألة التحسين تحتاج الآن إلى تهيئة الحالة الأساسية لـ $H_C$ (أو حالة ذات تداخل كبير معها) على الحاسوب الكمومي. ثم ستؤدي عملية أخذ العينات من هذه الحالة، باحتمالية عالية، إلى الحصول على حل لـ $\min~f(x)$.
الآن لنتأمل الهاميلتوني $H_C$ لمسألة **القطع الأقصى**. نربط كل قمة من الرسم البياني بكيوبت في الحالة $|0\rangle$ أو $|1\rangle$، حيث تدل القيمة على المجموعة التي تنتمي إليها القمة. الهدف من المسألة هو تعظيم عدد الحواف $(v_1, v_2)$ التي يكون فيها $v_1 = |0\rangle$ و$v_2 = |1\rangle$، أو العكس. إذا ربطنا العامل $Z$ بكل كيوبت، حيث

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

فإن الحافة $(v_1, v_2)$ تنتمي إلى القطع إذا كانت القيمة الذاتية لـ $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$؛ بمعنى آخر، يكون الكيوبتان المرتبطان بـ $v_1$ و$v_2$ مختلفين. وبالمثل، لا تنتمي $(v_1, v_2)$ إلى القطع إذا كانت القيمة الذاتية لـ $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$. لاحظ أننا لا نهتم بالحالة الكمومية الدقيقة المرتبطة بكل قمة، بل يهمنا فقط ما إذا كانت الحالتان متماثلتين أم مختلفتين عبر كل حافة. تتطلب مسألة القطع الأقصى إيجاد توزيع للكيوبتات على القمم بحيث تُقلَّل القيمة الذاتية للهاميلتوني التالي
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

بعبارة أخرى، $b_i = 0$ لجميع $i$ في مسألة القطع الأقصى. تمثل قيمة $Q_{ij}$ وزن الحافة. في هذا البرنامج التعليمي نعتبر رسماً بيانياً غير موزون، أي $Q_{ij} = 1.0$ لجميع $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### الخطوة 2: تحسين المسألة لتنفيذها على الأجهزة الكمومية
تحتوي الدائرة أعلاه على سلسلة من التجريدات المفيدة للتفكير في الخوارزميات الكمومية، غير أنه لا يمكن تشغيلها مباشرةً على الأجهزة. لتتمكن من التشغيل على وحدة معالجة كمومية QPU، يجب أن تخضع الدائرة لسلسلة من العمليات التي تشكّل خطوة **التحويل البرمجي** أو **تحسين الدائرة** في النمط.

توفر مكتبة Qiskit سلسلة من **ممرات التحويل البرمجي** التي تلبي نطاقاً واسعاً من تحويلات الدوائر. تحتاج إلى التأكد من أن دائرتك **محسَّنة** لغرضك المحدد.

قد ينطوي التحويل البرمجي على عدة خطوات، منها:

* **التعيين الأولي** للكيوبتات في الدائرة (كمتغيرات القرار) على الكيوبتات المادية في الجهاز.
* **فك التسلسل** للتعليمات في الدائرة الكمومية إلى تعليمات أصيلة للأجهزة يستطيع المعالج تنفيذها.
* **التوجيه** لأي كيوبتات في الدائرة تتفاعل مع كيوبتات مادية متجاورة.
* **قمع الأخطاء** بإضافة بوابات أحادية الكيوبت لقمع الضوضاء باستخدام فك الإقران الديناميكي.

مزيد من المعلومات حول التحويل البرمجي متاح في [التوثيق](/guides/transpile).

يحوّل الكود التالي الدائرة المجردة ويحسّنها إلى صيغة جاهزة للتنفيذ على أحد الأجهزة المتاحة عبر السحابة باستخدام **خدمة Qiskit IBM Runtime**.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![ناتج خلية الكود السابقة](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### الخطوة 3: التنفيذ باستخدام عوامل Qiskit الأولية
في سير عمل QAOA، تُحدَّد معاملات QAOA المثلى في حلقة تحسين تكرارية، تُشغّل سلسلة من تقييمات الدائرة وتستخدم محسّناً كلاسيكياً للعثور على المعاملات المثلى $\beta_k$ و$\gamma_k$. تُنفَّذ هذه الحلقة التنفيذية عبر الخطوات التالية:

1. تحديد المعاملات الأولية
2. إنشاء `Session` جديدة تحتوي على حلقة التحسين والعامل الأولي المستخدم لأخذ عينات من الدائرة
3. بعد العثور على المجموعة المثلى من المعاملات، تنفيذ الدائرة مرة نهائية للحصول على التوزيع الأخير الذي سيُستخدم في خطوة ما بعد المعالجة.
#### تحديد الدائرة بالمعاملات الأولية
نبدأ بمعاملات مختارة بشكل اعتباطي.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### تحديد المعالج الخلفي وعامل التنفيذ الأولي
استخدم **عوامل Qiskit Runtime الأولية** للتفاعل مع معالجات IBM&reg;. العاملان الأوليان هما Sampler وEstimator، ويعتمد الاختيار بينهما على نوع القياس الذي تريد إجراءه على الحاسوب الكمومي. لتقليل $H_C$، استخدم Estimator إذ إن قياس دالة التكلفة يعادل ببساطة القيمة المتوقعة لـ $\langle H_C \rangle$.
#### التشغيل
يوفر العاملان الأوليان مجموعة متنوعة من [أوضاع التنفيذ](/guides/execution-modes) لجدولة أحمال العمل على الأجهزة الكمومية، ويعمل سير عمل QAOA بشكل تكراري ضمن جلسة.

![توضيح يبيّن سلوك أوضاع التشغيل: المهمة المفردة والدُّفعة والجلسة.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

يمكنك توصيل دالة التكلفة المبنية على Sampler بروتين التقليل الخاص بـ SciPy للعثور على المعاملات المثلى.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![ناتج خلية الكود السابقة](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

بعد العثور على المعاملات المثلى للدائرة، يمكنك تعيين هذه المعاملات وأخذ عينات من التوزيع النهائي المحصَّل بالمعاملات المُحسَّنة. هنا يجب استخدام العامل الأولي *Sampler* إذ إن توزيع الاحتمالات لقياسات السلاسل البتية هو الذي يتوافق مع القطع الأمثل للرسم البياني.

**ملاحظة:** هذا يعني تهيئة حالة كمومية $\psi$ في الحاسوب ثم قياسها. سيؤدي القياس إلى انهيار الحالة إلى حالة أساسية حسابية واحدة – مثلاً `010101110000...` – تتوافق مع حل مرشح $x$ لمسألتنا التحسينية الأصلية ($\max f(x)$ أو $\min f(x)$ بحسب المهمة).

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### الخطوة 4: ما بعد المعالجة وإعادة النتيجة بالصيغة الكلاسيكية المطلوبة
تُفسِّر خطوة ما بعد المعالجة ناتج أخذ العينات لاستخلاص حل للمسألة الأصلية. في هذه الحالة، نهتم بالسلسلة البتية ذات أعلى احتمالية لأنها تحدد القطع الأمثل. تتيح التماثلات في المسألة أربعة حلول محتملة، وستعيد عملية أخذ العينات أحدها باحتمالية أعلى بقليل، غير أنك ستلاحظ في التوزيع المُرسَم أدناه أن أربع سلاسل بتية تبرز بوضوح بوصفها أكثر احتمالاً من بقية السلاسل.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### تصوير أفضل قطع
من خلال سلسلة البتات المثلى، يمكنك تصوير هذا القطع على الرسم البياني الأصلي.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

وحساب قيمة القطع:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## الجزء الثاني: توسيع النطاق!
لديك إمكانية الوصول إلى العديد من الأجهزة التي تحتوي على أكثر من 100 qubit على منصة IBM Quantum&reg;. اختر جهازاً لحل مسألة Max-Cut على رسم بياني موزون يضم 100 عقدة. هذه مسألة ذات "نطاق نفعي". خطوات بناء سير العمل هي نفسها المتبعة أعلاه، لكن مع رسم بياني أكبر بكثير.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### الخطوة 1: تحويل المدخلات الكلاسيكية إلى مسألة كمومية
#### الرسم البياني &rarr; Hamiltonian
أولاً، حوّل الرسم البياني الذي تريد حله مباشرةً إلى Hamiltonian مناسب لـ QAOA.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamiltonian &rarr; دائرة كمومية

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### الخطوة 2: تهيئة المسألة للتنفيذ الكمومي
للتوسع في خطوة تهيئة الدائرة لتشمل مسائل ذات نطاق نفعي، يمكنك الاستفادة من استراتيجيات التحويل عالية الأداء التي أُدخلت في Qiskit SDK الإصدار 1.0. تشمل الأدوات الأخرى خدمة المحوّل الجديدة مع [تمريرات المحوّل المُحسَّنة بالذكاء الاصطناعي](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

بمجرد العثور على المعاملات المثلى من تشغيل QAOA على الجهاز، قم بتعيين هذه المعاملات للدائرة.